In [2]:
from copy import *
from random import *
from jogos import *
from jogar import *
from utils import *
import string
import random

from collections import namedtuple

stateBreakthru = namedtuple('EstadoBreakthru', 'to_move, board_arg')

class EstadoBreakthru(stateBreakthru):
    
    def __init__(self, to_move=1, n=8, board_arg=[]):
        #self.terminou = self.verificar_terminou()
        if(len(board_arg) == 0):
            self.board = [['.' for i in range(n)] for i in range(n)]
            for i in range(2):
                for j in range(8):
                    self.board[i][j] = 'B'

            for i in range(6, 8):
                for j in range(8):
                    self.board[i][j] = 'W'
        else:
            self.board = board_arg

    def verificar_terminou(self):
        # 1 = White,  2 = Black
        if self.to_move == 2:
            for row in range(len(self.board)):
                if (self.board[0][row] == 'W'):
                    return 1
        elif self.to_move == 1:
            for row in range(len(self.board)):
                if (self.board[7][row] == 'B'):
                    return 2
        return 0
    
    def pos_convert(self, i, j):
        alfabeto = "abcdefgh"
        return alfabeto[j] + str(8 - i)

    def move_convert(self, move):
        numbers = []
        alfabeto = "abcdefgh"
        numbers.append(int(alfabeto.rfind(move[0]))) 
        numbers.append(int(move[1]) - 1)
        return numbers

    def moves(self, to_move):
        list_moves = []

        for row in range(len(self.board)):
            for col in range(len(self.board)):

                if to_move == 1 and self.board[col][row] == 'W': # para as brancas

                    if col > 0 and self.board[col-1][row] == '.': # cima
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col-1,row)))

                    if col > 0 and row < 7 and (self.board[col-1][row+1] == '.' or self.board[col-1][row+1] == 'B'): # diagonal direita cima
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col-1,row+1)))

                    if col > 0 and row > 0 and (self.board[col-1][row-1] == '.' or self.board[col-1][row-1] == 'B'): # diagonal esquerda cima
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col-1,row-1)))
                        
                if to_move == 2 and self.board[col][row] == 'B': # para as pretas

                    if col < 7 and self.board[col+1][row] == '.': # baixo
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col+1,row)))

                    if col < 7 and row < 7 and (self.board[col+1][row+1] == '.' or self.board[col+1][row+1] == 'W'): # diagonal direita baixo
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col+1,row+1)))

                    if col < 7 and row > 0 and (self.board[col+1][row-1] == '.' or self.board[col+1][row-1] == 'W'): # diagonal esquerda baixo
                        list_moves.append("{pos1}-{pos2}".format(pos1=self.pos_convert(col,row), pos2=self.pos_convert(col+1,row-1)))
    
        return sorted(list_moves)

## CLASS JOGO ##
class JogoBT_16(Game):
    """Play Breakthrough on an 8 x 8 board, with Max (first player) playing 'W' (1).
    """
    def __init__(self):
        self.initial = EstadoBreakthru(to_move=1, board_arg=[])

    def actions(self, state):
        "Legal moves for B and W"
        #Apenas queremos ver as ações possíveis para o jogador atual (to_move)
        return state.moves(state.to_move)

    def result(self, state, move):
        if state.to_move == 1:
            novo_estado = EstadoBreakthru(to_move=2, board_arg=state.board.copy)
            index_start = state.move_convert(move[0:2])
            new_index = state.move_convert(move[3:5])
            novo_estado.board[7 - index_start[1]][index_start[0]] = '.'
            novo_estado.board[7 - new_index[1]][new_index[0]] = 'W'
            return novo_estado
        else:
            novo_estado = EstadoBreakthru(to_move=1, board_arg=state.board.copy)
            index_start = state.move_convert(move[0:2])
            new_index = state.move_convert(move[3:5])
            novo_estado.board[7 - index_start[1]][index_start[0]] = '.'
            novo_estado.board[7 - new_index[1]][new_index[0]] = 'B'
            return novo_estado

    def utility(self, state, player):
        "If player wins in this state, return 1; if otherplayer wins return -1; else return 0."
        if player == state.verificar_terminou():
            return 1
        elif player != state.verificar_terminou() and state.verificar_terminou() > 0:
            return -1
        else:
            return 0

    def terminal_test(self, state):
        "A state is terminal if the whites reached line 8 or some black reached line 1."
        return state.verificar_terminou() >= 1

    def display(self, state):
        symbols = "abcdefgh"
        print("-"*17)
        for i in range(len(state.board)):
            print("{num}|".format(num=8-i) + ' '.join(state.board[i]))
        print("-+" + "-"*15)
        print(" |" + ' '.join(i for i in symbols))

        if state.verificar_terminou() >= 1:
            return
        elif state.to_move == 1:
            print("--NEXT PLAYER: W")
        elif state.to_move == 2:
            print("--NEXT PLAYER: B")

    def executa(self, estado, listaJogadas):
        "executa varias jogadas sobre um estado dado"
        "devolve o estado final "
        s = estado
        for j in listaJogadas:
            s = self.result(s, j)
        return s


In [3]:
accoes = "a2-a3 a7-a6 c2-c3 a6-b5 b2-b3 b5-b4".split()
jj = JogoBT_16()
interessante=jj.executa(jj.initial, accoes)
print("O que está lá?")
print(interessante.board[1][0])
novas_accoes = jj.actions(interessante)
outrointeressante=jj.result(interessante,novas_accoes[0])
print("O que está lá?")
print(interessante.board[1][0])
jj.display(interessante)
print("Jogadas possíveis: ", novas_accoes)

TypeError: object of type 'builtin_function_or_method' has no len()

In [11]:
accoes = "a2-a3 a7-a6 c2-c3 a6-b5 b2-b3 b5-b4".split()
jj = JogoBT_16()
interessante=jj.executa(jj.initial, accoes)
print("O que está lá?")
print(interessante.board[0][1])
novas_accoes = jj.actions(interessante)
outrointeressante=jj.result(interessante,novas_accoes[0])
print("O que está lá?")
print(interessante.board[0][1])
jj.display(interessante)
print("Jogadas possíveis: ", novas_accoes)

O que está lá?
B
O que está lá?
B
-----------------
8|B B B B B B B B
7|. B B B B B B B
6|. . . . . . . .
5|. . . . . . . .
4|. B . . . . . .
3|W W W . . . . .
2|W . . W W W W W
1|. W W W W W W W
-+---------------
 |a b c d e f g h
--NEXT PLAYER: W
Jogadas possíveis:  ['a1-a2', 'a1-b2', 'a3-a4', 'a3-b4', 'b1-a2', 'b1-b2', 'b1-c2', 'b3-a4', 'b3-c4', 'c1-b2', 'c1-c2', 'c3-b4', 'c3-c4', 'c3-d4', 'd1-c2', 'd2-d3', 'd2-e3', 'e2-d3', 'e2-e3', 'e2-f3', 'f2-e3', 'f2-f3', 'f2-g3', 'g2-f3', 'g2-g3', 'g2-h3', 'h2-g3', 'h2-h3']


In [12]:
accoes = "a2-a3 a7-a6 c2-c3 a6-b5 b2-b3 b5-b4"
actions = ["g2-h3","e7-f6"]
['a2-a3', 'a2-b3', 'b2-a3', 'b2-b3', 'b2-c3', 'c2-b3', 'c2-c3', 'c2-d3', 'd2-c3', 'd2-d3', 'd2-e3', 'e2-d3', 'e2-e3', 'e2-f3', 'f2-e3', 'f2-f3', 'f2-g3', 'g2-f3', 'g2-g3', 'g2-h3', 'h2-g3', 'h2-h3']

def test(accoes):
    for i in range(1, 9):
        for mov in accoes:
            if str(i) in accoes[0:2]:
                return mov

print(test(accoes))
    
    





a


In [ ]:
umsetnormal = set()
umsetnormal.add("a1-a2")
umsetnormal.add("b1-b2")
for x in umsetnormal:
    print("1" in x[1])

True
True


In [ ]:
moves = set()
moves.add("b1-b2")
moves.add("a1-a2")
for i in range(1, 9):
        for mov in moves:
            if str(i) in mov[1]:
                print(mov)

a1-a2
b1-b2
